In [1]:
import pandas as pd
import librosa
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler

In [12]:
model_dir = 'trained_models/speakerGender_KaggleVAD/'

# Function to extract features (same as before)
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr, n_fft=512).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_fft=512).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr, n_fft=512).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr).T, axis=0)
    return np.hstack([mfccs, chroma, mel, contrast, tonnetz])

# Example: Load a model and make a prediction
model_name = 'Logistic Regression'  # Choose the model you want to use
model = joblib.load(os.path.join(model_dir, f"{model_name}.pkl"))

In [22]:
#sample_file = '../../LibriVox_Kaggle/achtgesichterambiwasse/achtgesichterambiwasse_0000.wav'
#sample_file = '../FSMN-VAD/asr_example.wav'
sample_file = '../audioData/VoxConverse/audio/hqyok.wav'

features = extract_features(sample_file)
scaler = joblib.load(os.path.join(model_dir, 'scaler.pkl'))
features = scaler.transform([features])

In [23]:
# Make preds
prediction = model.predict(features)
encoder = joblib.load(os.path.join(model_dir, 'label_encoder.pkl'))
encoder.inverse_transform(prediction)

array(['Male'], dtype=object)